# FIFA API: World Cup players/teams

#### Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_stiles as altstiles
import numpy as np
import requests
import json

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('stiles')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [5]:
today_str = pd.Timestamp("today").strftime("%m/%d/%Y")
today = pd.Timestamp("today")

---

## Get data

#### Read teams

In [6]:
teams_df = pd.read_csv("data/processed/world_cup_teams.csv", dtype={"idteam": str})

In [7]:
teams_df.head()

,idteam,idconfederation,shortclubname,abbreviation,pictureurl
0,43922,CONMEBOL,Argentina,ARG,https://cloudinary.fifa.com/api/v3/picture/fla...
1,43976,AFC,Australia,AUS,https://cloudinary.fifa.com/api/v3/picture/fla...
2,43935,UEFA,Belgium,BEL,https://cloudinary.fifa.com/api/v3/picture/fla...
3,43924,CONMEBOL,Brazil,BRA,https://cloudinary.fifa.com/api/v3/picture/fla...
4,43849,CAF,Cameroon,CMR,https://cloudinary.fifa.com/api/v3/picture/fla...


#### Create a dict to map team ids to full names

In [8]:
team_id_to_name = pd.Series(
    teams_df.shortclubname.values, index=teams_df.idteam
).to_dict()

In [9]:
teams_list = teams_df["idteam"].to_list()

---

#### Get player details from FIFA API

In [10]:
player_jsons = []
players_list = []

for t in teams_list:
    player_response = requests.get(
        f"https://api.fifa.com/api/v3/teams/{t}/squad?idCompetition=17&idSeason=255711&language=en"
    )
    for j in player_response.json()["Players"]:
        player_dict = {
            "player_id": j["IdPlayer"],
            "name": j["PlayerName"][0]["Description"],
            "position": j["Position"],
            "weight": j["Weight"],
            "height": j["Height"],
            "country": j["IdCountry"],
            "country_id": j["IdTeam"],
            "number": j["JerseyNum"],
            "birthday": j["BirthDate"],
            "picture": j["PlayerPicture"]["PictureUrl"],
        }
        players_list.append(player_dict)

In [11]:
# player_response.json()["Players"]

In [12]:
players_df = pd.DataFrame(players_list).sort_values("height", ascending=False)

#### Clean up a bit

In [13]:
players_df["number"] = players_df["number"].astype(str)

In [14]:
players_df["name"] = players_df["name"].str.title()

#### Ages

In [15]:
players_df["dob"] = pd.to_datetime(players_df["birthday"]).dt.strftime("%Y-%m-%d")

In [16]:
players_df["age"] = (
    ((pd.to_datetime(today_str) - pd.to_datetime(players_df["dob"])).dt.days) / 365.25
).round(2)

In [17]:
players_df = players_df.drop(["birthday"], axis=1)

#### Add country names

In [18]:
players_df["country_name"] = players_df["country_id"].map(team_id_to_name)

#### Positions

In [19]:
players_df["position"] = players_df["position"].astype(str)

In [20]:
positions = {"0": "Goalkeeper", "1": "Defender", "2": "Midfielder", "3": "Striker"}

In [21]:
players_df["position_desc"] = players_df["position"].map(positions)

---

## Aggregate

#### Mean height, weight and age, by team

In [22]:
players_grouped = (
    players_df.groupby(["country_name"])
    .agg({"weight": "mean", "height": "mean", "age": "mean"})
    .round(2)
    .reset_index()
)

#### Tallest

In [23]:
players_grouped.sort_values("height", ascending=False).head()

,country_name,weight,height,age
25,Serbia,78.46,187.19,27.44
8,Denmark,82.00,186.12,27.76
11,France,82.54,185.46,27.19
24,Senegal,76.00,184.92,26.54
12,Germany,80.04,184.73,27.40


#### Oldest

In [24]:
players_grouped.sort_values("age", ascending=False).head()

,country_name,weight,height,age
14,IR Iran,79.00,183.72,29.50
17,Mexico,74.12,178.81,29.05
28,Tunisia,76.96,182.69,28.62
3,Brazil,74.85,180.77,28.51
2,Belgium,78.46,184.65,28.43


---

## USA

In [25]:
usa_players_df = players_df.query('country == "USA"')

In [26]:
usa_players_df.head()

,player_id,name,position,weight,height,country,country_id,number,picture,dob,age,country_name,position_desc
764,385917,Ethan Horvath,0,84.0,193.0,USA,43921,12,https://digitalhub.fifa.com/transform/f3b199e6...,1995-06-09,27.53,USA,Goalkeeper
771,390276,Haji Wright,3,79.0,191.0,USA,43921,19,https://digitalhub.fifa.com/transform/f1ee9fad...,1998-03-27,24.73,USA,Striker
753,448217,Matt Turner,0,79.0,190.0,USA,43921,1,https://digitalhub.fifa.com/transform/402cd2c6...,1994-06-24,28.49,USA,Goalkeeper
755,369097,Walker Zimmerman,1,84.0,190.0,USA,43921,3,https://digitalhub.fifa.com/transform/388db833...,1993-05-19,29.59,USA,Defender
777,305832,Sean Johnson,0,91.0,190.0,USA,43921,25,https://digitalhub.fifa.com/transform/61a88ddb...,1989-05-31,33.56,USA,Goalkeeper


---

#### No. 10s

In [32]:
players_df.query('number == "10"').head()

,player_id,name,position,weight,height,country,country_id,number,picture,dob,age,country_name,position_desc
269,336043,Raheem Sterling,3,70.0,170.0,ENG,43942,10,https://digitalhub.fifa.com/transform/e593063f...,1994-12-08,28.03,England,Striker
9,229397,Lionel Messi,3,72.0,170.0,ARG,43922,10,https://digitalhub.fifa.com/transform/40e6d6b5...,1987-06-24,35.49,Argentina,Striker
606,339745,Salem Aldawsari,3,70.0,172.0,KSA,43835,10,https://digitalhub.fifa.com/transform/072ede2c...,1991-08-19,31.34,Saudi Arabia,Striker
191,241559,Luka Modric,2,66.0,172.0,CRO,43938,10,https://digitalhub.fifa.com/transform/d67acdb0...,1985-09-09,37.28,Croatia,Midfielder
554,395205,Bernardo Silva,3,65.0,173.0,POR,43963,10,https://digitalhub.fifa.com/transform/c63cbee9...,1994-08-10,28.36,Portugal,Striker


In [28]:
players_df = players_df.sort_values("height")

In [29]:
players_df.head()

,player_id,name,position,weight,height,country,country_id,number,picture,dob,age,country_name,position_desc
479,433710,Ilias Chair,2,64.0,164.0,MAR,43872,13,https://digitalhub.fifa.com/transform/02b330d8...,1997-10-30,25.14,Morocco,Midfielder
122,401789,Collins Fai,1,70.0,165.0,CMR,43849,19,https://digitalhub.fifa.com/transform/0373615a...,1992-08-13,30.35,Cameroon,Defender
412,430405,Yuki Soma,2,68.0,166.0,JPN,43819,24,https://digitalhub.fifa.com/transform/923ad90a...,1997-02-25,25.82,Japan,Midfielder
458,251352,Andres Guardado,2,63.0,167.0,MEX,43911,18,https://digitalhub.fifa.com/transform/9aeabab1...,1986-09-28,36.23,Mexico,Midfielder
461,403586,Uriel Antuna,3,62.0,167.0,MEX,43911,21,https://digitalhub.fifa.com/transform/fd0fef42...,1997-08-21,25.33,Mexico,Striker


---

## Exports

In [30]:
players_df.to_csv("data/processed/world_cup_players.csv", index=False)
usa_players_df.to_csv("data/processed/world_cup_teams_usa.csv", index=False)
players_df.query('number == "10"').to_csv(
    "data/processed/world_cup_players_number10s.csv", index=False
)